In [5]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import tweepy, csv, time, re
import numpy as np
from IPython.display import clear_output
import operator
import pandas as pd
import json

# Firebase API Keys and Constants
SERVICE_ACCOUNT_PATH = "firebase_secrets.json"
EVENT_ID = 'dTUBW5yy1PZJouA45rsP'

# Twitter API Keysand Constants
keys = json.load(open('keys.json'))
consumer_key = keys.get('consumer_key')
consumer_secret = keys.get('consumer_secret')
access_token = keys.get('access_token')
access_token_secret = keys.get('access_token_secret')
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
analyser = SentimentIntensityAnalyzer()

In [6]:
cred = credentials.Certificate(SERVICE_ACCOUNT_PATH)
try:
    firebase_admin.initialize_app(cred)
except:
    pass

db = firestore.client()

# Add a new doc in collection 'cities' with ID 'LA'
def write_to_firebase(scores):
    db.collection('events').document(EVENT_ID).collection('scores').document(str(int(time.time()))).set(scores)

In [7]:
class Person:
    
    def __init__(self, key, name, keywords=None):
        self.key = key
        self.name = name
        
        if keywords == None:
            self.keywords = [self.key, self.name]
        else:
            self.keywords = keywords
            
        self.score = 0.0
        self.cumscore = 0.0

In [8]:
people = []
people.append(Person(key='Bennet', name='Michael Bennet'))
people.append(Person(key='Gillibrand', name='Kirsten Gillibrand'))
people.append(Person(key='Castro', name='Julián Castro'))
people.append(Person(key='Booker', name='Cory Booker'))
people.append(Person(key='Biden', name='Joe Biden'))
people.append(Person(key='Harris', name='Kamala Harris'))
people.append(Person(key='Yang', name='Andrew Yang'))
people.append(Person(key='Gabbard', name='Tulsi Gabbard'))
people.append(Person(key='Inslee', name='Jay Inslee'))
people.append(Person(key='Blasio', name='Bill de Blasio'))

person_map = {}
keyword_map = {}
for p in people:
    person_map[p.key] = p
    for key in p.keywords:
        keyword_map[key] = p.key
keyword_map.pop('Yang')

'Yang'

In [9]:
def get_scoreboard():
    scores = {}
    for person in people:
        scores[person.key] = person.score
    return scores

def get_individual_scoreboard():
    scores = {}
    for person in people:
        scores[person.key] = person.cumscore
    return scores
        
def print_scoreboard(i):
    clear_output()
    x = get_individual_scoreboard()
    print("Tweets processed: " + str(i))
    df = [[key, value] for key, value in x.items()]
    board = pd.DataFrame(df, columns=['Candidate', 'Score']).sort_values(by=['Score'], ascending=False).reset_index()
    print(board.loc[:, 'Candidate':])
    
    
def reset_cum_scoreboard():
    for person in people:
        person.cumscore = 0
        
def reset_scoreboard():
    for person in people:
        person.cumscore += person.score
        person.score = 0

In [10]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r: input_txt = re.sub(i, '', input_txt)        
    return input_txt

def clean(tweet):
    patterns = ["RT @[\w]*:", "@[\w]*", "https?://[A-Za-z0-9./]*"]
    for pattern in patterns:
        tweet = remove_pattern(tweet, pattern)
    tweet = re.sub('[^a-z A-Z#]', ' ', tweet)
    return tweet

In [11]:
i = 0
def process(tweet):
    global i
    i += 1
    
    found = []
    for keyword in keyword_map.keys():
        if keyword.lower() in tweet.lower():
            found.append(keyword_map[keyword])
    if len(found) == 1:
        person_map[found[0]].score = person_map[found[0]].score + calculate_score(tweet)
        
        
    if i % 200 == 0:
        print_scoreboard(i)
        reset_scoreboard()
    
    if i % 10000 == 0:
        reset_cum_scoreboard()
    
    if i % 500 == 0:
        write_to_firebase(get_scoreboard())

In [12]:
def calculate_score(tweet):
    tweet = clean(tweet)
    score = analyser.polarity_scores(tweet)['compound']
    return score

In [13]:
def twitter_stream_listener(file_name, filter_track, follow=None,
                            locations=None, languages=None):
    
    class CustomStreamListener(tweepy.StreamListener):
        
        def __init__(self):
            super(CustomStreamListener, self).__init__()
            
        def on_status(self, status):
            process(status.text)
            
        def on_error(self, status_code):
            
            if status_code == 420:
                print('Encountered error code 420. Disconnecting the stream')
                # Returning False in on_data disconnects the stream
                return False
            else:
                print('Encountered error with status code, resuming automatically: {}'.format(
                    status_code))
                return True  # Don't kill the stream
            
        def on_timeout(self):
            print('Timeout...resuming stream')
            return True  # Don't kill the stream
        
    print("Starting streaming...")
    
    streamingAPI = tweepy.streaming.Stream(auth, CustomStreamListener())
    streamingAPI.filter(track=filter_track, follow=follow, locations=locations, languages=languages)

In [15]:
# file_name = 'test.csv'
# twitter_stream_listener(file_name, keyword_map.keys())